<a href="https://colab.research.google.com/github/DerekChuang/tensorflow_practice/blob/master/dataset_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import tensorflow as tf
import numpy as np
import shutil

In [0]:
shutil.rmtree("outdir", ignore_errors = True) # start fresh each time


In [25]:
'''
從桌機上載data

#上載 2.csv
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# 確認 2.csv 是否已上載
!ls *.* -l


# use Pandas to read 2.csv
import pandas as pd
df = pd.read_csv('2.csv')
print(df)

'''

'\n從桌機上載data\n\n#上載 2.csv\nfrom google.colab import files\n\nuploaded = files.upload()\n\nfor fn in uploaded.keys():\n  print(\'User uploaded file "{name}" with length {length} bytes\'.format(\n      name=fn, length=len(uploaded[fn])))\n\n# 確認 2.csv 是否已上載\n!ls *.* -l\n\n\n# use Pandas to read 2.csv\nimport pandas as pd\ndf = pd.read_csv(\'2.csv\')\nprint(df)\n\n'

In [26]:
#掛載 google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
!ls 


drive  sample_data


In [28]:
!ls '/content/drive/My Drive/Colab Notebooks'




 “basic_classification.ipynb”		   dataset_train_eval.ipynb
 “basic-text-classification.ipynb”的副本  'numpy&pandas_input.ipynb'
 constant_input.ipynb			   property_prices_01.csv
 Dataset_API_intro.ipynb		   property_prices_02.csv


In [0]:
#filenames_dataset = tf.data.Dataset.list_files('/content/drive/My Drive/Colab Notebooks/property_prices_01.csv')


In [0]:
#filenames_dataset

In [0]:
def decode_csv(row):
    cols = tf.decode_csv(row, record_defaults=[[0],['house'],[0]])
    features = {'sq_footage': cols[0], 'type': cols[1]}
    price = cols[2] # "label"
    return features, price

dataset = tf.data.Dataset.list_files('/content/drive/My Drive/Colab Notebooks/property*') \
                         .flat_map(tf.data.TextLineDataset)  \
                         .map(decode_csv)
        
dataset = dataset.shuffle(1000) \
                 .repeat()  \
                 .batch(3)
        
featcols = [
    tf.feature_column.numeric_column("sq_footage"),
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list("type", ["house", "apt"])
    )
]
        


@staticmethod
list_files(
    file_pattern,
    shuffle=None,
    seed=None
)



Example: If we had the following files on our filesystem: - /path/to/dir/a.txt - /path/to/dir/b.py - /path/to/dir/c.py If we pass "/path/to/dir/*.py" as the directory, the dataset would produce: - /path/to/dir/b.py - /path/to/dir/c.py

flat_map(map_func)
Maps map_func across this dataset and flattens the result.

为了对每个文件进行处理，可以使用Dataset.flat_map()为每个文件创建嵌套的数据集:

使用tf.data.TextLineDataset可以从单个或多个文本文件中逐行的读取数据。

Dataset.map(f)处理数据中的每一个元素element，其中函数f(element) 需要返回一个新的元素element。

tf.decode_csv(
    records,
    record_defaults,
    field_delim=',',
    use_quote_delim=True,
    name=None,
    na_value='',
    select_cols=None
)


tf.feature_column.indicator_column
tf.feature_column.indicator_column(categorical_column)
Defined in tensorflow/python/feature_column/feature_column.py.

Represents multi-hot representation of given categorical column.

For DNN model, indicator_column can be used to wrap any categorical_column_* (e.g., to feed to DNN). Consider to Use embedding_column if the number of buckets/unique(values) are large.

For Wide (aka linear) model, indicator_column is the internal representation for categorical column when passing categorical column directly (as any element in feature_columns) to linear_model. See linear_model for details.

In [0]:
def train_input_fn():
    return dataset.make_one_shot_iterator().get_next()

def serving_input_fn():
    json = { 'sq_footage' : tf.placeholder(tf.int32, [None]),
             'prop_type'  : tf.placeholder(tf.string, [None])
           }
        
    features = { 'sq_footage': json['sq_footage'],
                 'type': json['prop_type']
               }
    return tf.estimator.export.ServingInputReceiver(features, json)



In [32]:
# try to read the Dataset by hand
features, labels = train_input_fn()
with tf.Session() as sess:
    for i in range(40):
        feat, lab = sess.run([features, labels])
        print("feat:" + str(feat) + " label:" + str(lab))

feat:{'sq_footage': array([1101, 2201, 2101], dtype=int32), 'type': array([b' apt', b' apt', b' apt'], dtype=object)} label:[ 726 1351 1326]
feat:{'sq_footage': array([1001, 1000, 2001], dtype=int32), 'type': array([b' house', b'house', b' apt'], dtype=object)} label:[ 501  500 1301]
feat:{'sq_footage': array([1100, 2200, 2000], dtype=int32), 'type': array([b'house', b'apt', b'house'], dtype=object)} label:[ 525 1350 1000]
feat:{'sq_footage': array([3101, 1101, 3201], dtype=int32), 'type': array([b' house', b' house', b' apt'], dtype=object)} label:[1526  526 1951]
feat:{'sq_footage': array([1200, 1001, 3000], dtype=int32), 'type': array([b'house', b' apt', b'apt'], dtype=object)} label:[ 550  701 1900]
feat:{'sq_footage': array([2100, 2000, 2101], dtype=int32), 'type': array([b'house', b'apt', b' house'], dtype=object)} label:[1025 1300 1026]
feat:{'sq_footage': array([1200, 3201, 3001], dtype=int32), 'type': array([b'apt', b' house', b' house'], dtype=object)} label:[ 750 1551 1501]


你需要做四件事。
 
選擇您的估算器，提供運行配置，
 
並通過TrainSpec和EvalSpec提供培訓和測試數據。


In [0]:
# run _ config tells the estimator where and how often 
# to write Checkpoints and Tensorboard logs 



run_config = tf.estimator.RunConfig(model_dir="outdir",
                                    save_summary_steps=10,
                                    save_checkpoints_steps=1000)
        


In [0]:
export_last = tf.estimator.LatestExporter("houseprice",
                                          serving_input_fn)
        


In [0]:
# The TrainSpec tells the estimator how to get 
# training data 


train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=5000)
        


In [0]:
# The EvalSpec controls the evaluation and the 
# checkpointing of the model since they happen at the same time 



eval_spec = tf.estimator.EvalSpec(input_fn=train_input_fn, ## use real evaldata !
                                  steps=10, # evals on 100 batches
                                  throttle_secs=3, # eval no more than every 3 sec
                                  exporters=export_last)



# train_and_evaluate

In [37]:
#model = tf.estimator.LinearRegressor(featcols, config=run_config)
model = tf.estimator.DNNRegressor([3, 2, 2], featcols, config=run_config)

tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Using config: {'_model_dir': 'outdir', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f951e6a7470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will

({'average_loss': 1526493.6,
  'global_step': 5000,
  'label/mean': 1147.2,
  'loss': 4579481.0,
  'prediction/mean': 6.8027096},
 [b'outdir/export/houseprice/1540886706'])

In [38]:
! ls

drive  outdir  sample_data
